In [ ]:
import shutil
import pandas as pd
import numpy as np
from scipy import stats
import os
import matplotlib.pyplot as plt
from matplotlib import style
import datetime as dt
from IPython.display import Audio, display

pd.options.mode.chained_assignment = None   # turn off warnings

## Import of Delay Data

In [ ]:
# === list one for DD-MM files
listpaths1 = [
    'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/152051e6-1193-43c0-89b8-21b361f49970/download/fahrzeiten_soll_ist_20180401_20180407.csv',
#     'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/eb403fd1-8f8b-475e-98aa-f04ee3b255ba/download/fahrzeiten_soll_ist_20180311_20180317.csv']
    'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/97e59d2a-83ec-438f-ae6f-0fe85d9bc1e6/download/fahrzeiten_soll_ist_20180304_20180310.csv',
#     'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/a38c5d0f-b732-4f5a-9786-eb01a2ffa0bb/download/fahrzeiten_soll_ist_20180211_20180217.csv']
    'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/f17a950d-5be5-4b00-bafd-3c859afcc6cc/download/fahrzeiten_soll_ist_20180204_20180210.csv']
#     'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/bf39176c-d6c0-4741-8cd5-4e46e45c608e/download/fahrzeiten_soll_ist_20180107_20180113.csv']
 
frame1 = pd.DataFrame()
list_1 = []
for path_ in listpaths1:
    df1 = pd.read_csv(path_, usecols=[0,1,2,10,11], parse_dates=['betriebsdatum'] ,infer_datetime_format=True,  index_col=None)
    list_1.append(df1)
df1 = pd.concat(list_1)

# === change datime from DD-MM to MM-DD
df1['betriebsdatum'] = pd.to_datetime(df1['betriebsdatum'])
df1['time'] = df1['betriebsdatum'].dt.strftime('%Y-%d-%m')
df1['time'] = pd.to_datetime(df1['time'], format='%Y-%m-%d', errors='ignore')
# df1.drop(df1['betriebsdatum'], axis=, inplace=True)

# df1['betriebsdatum'] = pd.to_datetime(df1['betriebsdatum'], format='%Y/%m/%d', errors='ignore')
# df1['time'] = df1['betriebsdatum'].dt.strftime('%Y/%d/%m')
# df1['time'] = pd.to_datetime(df1['time'], format='%Y-%m-%d', errors='ignore')

# === list two for MM-DD files
listpaths2 = [
    'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/03ec9d0a-b16f-4e78-8e4f-2da4970efbb6/download/fahrzeiten_soll_ist_20180325_20180331.csv',
    'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/c88a3801-c6fc-4d32-8ece-e269899be497/download/fahrzeiten_soll_ist_20180318_20180324.csv',
    'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/a265b5d8-287f-4d22-88b2-f3a1770e1a4a/download/fahrzeiten_soll_ist_20180225_20180303.csv',
    'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/1ac13127-fcde-4ac2-8462-50f348fd28fe/download/fahrzeiten_soll_ist_20180218_20180224.csv',
    'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/ee7f8901-5466-425d-98e6-98ddcf849079/download/fahrzeiten_soll_ist_20180128_20180203.csv',
    'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/b45b383e-4b0d-4ad0-8bee-e958c5e7360a/download/fahrzeiten_soll_ist_20180121_20180127.csv',
    'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/4bbaf516-824a-41b3-8a95-6679745ce7d3/download/fahrzeiten_soll_ist_20180114_20180120.csv',
    'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/resource/b4e64312-e7f6-4b43-a832-1a8e47138732/download/fahrzeiten_soll_ist_20171231_20180106.csv']

frame2 = pd.DataFrame()
list_2 = []
for path_ in listpaths2:
    df2 = pd.read_csv(path_, usecols=[0,1,2,10,11], parse_dates=['betriebsdatum'] ,infer_datetime_format=True, index_col=None)
    list_2.append(df2)
df2 = pd.concat(list_2)

df2['time'] = pd.to_datetime(df2['betriebsdatum'])

# === put both together
frames = [df1, df2]
df = pd.concat(frames)

# === check status
check = pd.DataFrame(data=df)   # check size
col = check.shape[1]
row = check.shape[0]
print('delay data rows: %d' % row)
print('delay data columns: %d' % col)
print(df.head(3))
print(df.dtypes)

display(Audio("./weather/ape.mp3", autoplay=True))

## Time difference calculation and formatting

In [ ]:
def mask(df, key, value):
    return df[df[key] == value]

pd.DataFrame.mask = mask
df3 = df.mask('linie', 69)

df3.loc[:, 'diff'] = (df3['ist_an_von'] - df3['soll_an_von'])
df4 = df3.copy()
df4.reset_index(drop=True, inplace=True)
df4.loc[:, 'zeit'] = df4.loc[:, 'soll_an_von'].copy().astype(float)
df4.loc[:, 'zeit'] = pd.to_datetime(df4.loc[:, 'zeit'], errors='coerce', unit='s')
df4.loc[:, 'zeit'] = df4.loc[:, 'zeit'].dt.strftime('%H:%M')
df4.loc[:, 'zeit'] = pd.to_datetime(df4.loc[:, 'zeit'], format='%h:%m', errors='ignore')
df5 = df4.copy()
df5.drop(df5.columns[[3, 4]], axis=1, inplace=True)
df6 = df5.copy()
df6['time'] = df6['time'].dt.strftime('%Y%m%d')
df6['time'] = pd.to_datetime(df6['time'] + ' ' + df5['zeit'])
df7 = df6.copy()
df7.drop(df7.columns[[2]], axis=1, inplace=True)
df7.loc[:, 'diff'] = df7.loc[:, 'diff'].apply(pd.to_numeric, errors='coerce', downcast='float')
df7.loc[:, 'time'] = df7.loc[:, 'time'].dt.round('60min')
df7 = df7.dropna(how='any')
df7['diff'] = df7['diff'].astype(int)
df7 = df7.loc[df7['diff'] > 0]
df8 = df7.copy()
df8.drop(df8.columns[[4]], axis=1, inplace=True)
df8['diff'] = df8['diff'] / 60    # from seconds to minutes
df8['weekday'] = df8['time'].dt.dayofweek

# === check size
check = pd.DataFrame(data=df6)
col = check.shape[1]
row = check.shape[0]
print('cleaned diff: %d' % row)
print('cleaned diff: %d' % col)
print(df4.head())
print(df8.head(4))
print(df8.dtypes)

display(Audio("./weather/ape.mp3", autoplay=True))

## Weather Data

In [ ]:
# === input folder
path = r'./weather/*.csv'
# === import csv as dataframe
new_cols = ['weather']
we = pd.read_csv('./weather/agrometeo-data.csv', encoding='Latin-1', header=None, names=new_cols)
wet1 = pd.DataFrame(data=we)
# === clean-up
wet = wet1.iloc[3:]
wet.loc[:, 'time'] = wet.weather.str.split(';').str.get(0)
wet.loc[:, 'temp'] = wet.weather.str.split(';').str.get(1)
wet.loc[:, 'rain'] = wet.weather.str.split(';').str.get(2)
wet.drop(wet.columns[[0]], axis=1, inplace=True)
wet.loc[:, 'time'] = pd.to_datetime(wet.loc[:, 'time'])
wet.loc[:, 'rain'] = wet.loc[:, 'rain'].apply(pd.to_numeric, errors='coerce')
wet = wet.dropna(how='any')

# === check size
q = wet.shape[1]
o = wet.shape[0]
print('weather data rows: %d' % o)
print('weather data columns: %d' % q)
print(wet.head(3))
print(wet.dtypes)

## Matching Rain and Time Difference

In [ ]:
# === left merge on both time col which are in datetime format
merge = df8.merge(wet, left_on='time', right_on='time', how='left')

# === check size                         
print(merge.head())
print(merge.dtypes)

## Smooth out the data

In [ ]:
smooth = merge.copy()
q = smooth['diff'].quantile(0.99)
smooth = smooth[smooth['diff'] <= q]
print(q)
smooth = smooth.dropna(how='any')
style.use('ggplot')
scurr = smooth.copy()
scurr['hour'] = scurr.loc[:, 'time'].apply(lambda x: x.hour)
scurr.drop(scurr.columns[[0, 1]], axis=1, inplace=True)
print(scurr.head())
# === frequency histogramm
plt.hist(smooth.loc[:, 'diff'])
plt.title("delay frequency")
plt.xlabel("delay[minutes]")
plt.ylabel("frequency")
plt.savefig('./output/2Dfrequency.png', format='png', dpi=800)
plt.show()

## Correlation

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

style.use("ggplot")

# === output folder
newpath = r'./output/'
if os.path.exists(newpath):
    shutil.rmtree(newpath, ignore_errors=True)
os.makedirs(newpath)

month = smooth.copy()
day = smooth.copy()
hour = smooth.copy()

month['time'] = month.loc[:, 'time'].apply(lambda x: x.month)
day['time'] = day.loc[:, 'time'].apply(lambda x: x.day)
hour['time'] = hour.loc[:, 'time'].apply(lambda x: x.hour)

# === csv to check output data
# merge.to_csv('./output/merge.csv', header=False, index=False)
# wet.to_csv('./output/wet.csv', header=False, index=False)
# df1.to_csv('./output/df1.csv', header=False, index=False)

# === 2D monthly distribution of delays
x = month.loc[:, 'time']
y = month.loc[:, 'diff']
plt.scatter(x, y)
plt.title("monthly distribution")
plt.xlabel("months")
plt.ylabel("delay[minutes]")
plt.savefig('./output/2Dmonthly.png', format='png', dpi=800)
# plt.show()

# === 2D daily distribution of delays
x = day.loc[:, 'weekday']
y = day.loc[:, 'diff']
plt.scatter(x, y)
plt.title("daily distribution")
plt.xlabel("weekday")
plt.ylabel("delay[minutes]")
plt.savefig('./output/2Ddaily.png', format='png', dpi=800)
# plt.show()

# === 2D hourly distribution of delays
x = hour.loc[:, 'time']
y = hour.loc[:, 'diff']
plt.scatter(x, y)
plt.title("hourly distribution")
plt.xlabel("hours")
plt.ylabel("delay[minutes]")
plt.savefig('./output/2Dhourly.png', format='png', dpi=800)
# plt.show()

# === 2D dependency on rain
x = smooth.loc[:, 'rain']
y = smooth.loc[:, 'diff']
plt.scatter(x, y)
plt.title("correlation")
plt.xlabel("rain[mm/m2]")
plt.ylabel("delay[minutes]")
plt.savefig('./output/2Drain-delay.png', format='png', dpi=800)
# plt.show()

# === 3D Figure rain, time, delay
x = hour.loc[:, 'time']
y = hour.loc[:, 'rain']
z = hour.loc[:, 'diff']
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(x, y, z)
plt.title("Correlation")
ax.set_xlabel('hours')
ax.set_ylabel('rain [mm/m2]')
ax.set_zlabel('delay[minutes]')
plt.savefig('./output/3DcorrelationHourly.png', format='png', dpi=800)
# plt.show()

# === other statistics on data

print(smooth.loc[:,'rain'].describe())
print(smooth.loc[:,'diff'].describe())

slope, intercept, r_value, p_value, std_err = stats.linregress(smooth.loc[:, 'rain'], smooth.loc[:, 'diff'])
print("r-squared value between rain and delay:", r_value ** 2)

display(Audio("./weather/ape.mp3", autoplay=True))

## Prediction

In [ ]:
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn import cross_validation, linear_model
import pickle

dataframe = scurr
array = dataframe.values
X = array[:,[2,3,4,5]]
Y = array[:,[1]].ravel()
Y = Y.astype('int')
test_size = 0.33
seed = 2
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

plt.scatter(X[:, 0], X[:, 2])
plt.xlabel('weekday')
plt.ylabel('precipitation[mm/m2]')
plt.title('precipitation & weekday')
plt.savefig('./output/weekdays&rain.png', format='png', dpi=800)
plt.show()

loo = cross_validation.LeaveOneOut(len(Y))
# its going down
model = SVC(kernel = 'linear', C = 1.0, decision_function_shape='ovo')
# model = LogisticRegression()
model.fit(X_train, Y_train)

# save the model to disk
filename = 'first_prediction.sav'
pickle.dump(model, open(filename, 'wb'))

scores = cross_validation.cross_val_score(model, X_digits, Y_digits, scoring='neg_mean_squared_error', cv=loo,)
print('Mean score from cross validation:', scores.mean())
# print('CrossValScore:', cross_val_score(model, X_train, Y_train, scoring='neg_mean_squared_error'))

# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

pred = model.predict(X_test)

for i in range(5):
    print("X=%s, Predicted=%s" % (X_test[i], Y_test[i]))

display(Audio("./weather/ape.mp3", autoplay=True))